#INTRO
This notebook imports files from an s3 bucket and performs some basic nlp tasks

In [11]:
!pip3 uninstall spacy -y
!pip3 uninstall textacy -y


Usage:   
  pip3 uninstall [options] <package> ...
  pip3 uninstall [options] -r <requirements file> ...

no such option: -U

Usage:   
  pip3 uninstall [options] <package> ...
  pip3 uninstall [options] -r <requirements file> ...

no such option: -U


In [1]:
# Install spaCy 
!pip3 install -U spacy==3.3.0

# Download the large English model for spaCy
!python3 -m spacy download en_core_web_lg

# Install textacy which will also be useful
#!pip3 install -U textacy

     |████████████████████████████████| 400.7 MB 1.5 kB/s             
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [2]:
from sagemaker import get_execution_role
import boto3
import spacy

In [3]:
bucket='cogni-nlp-bucket'
sub='nlp'

conn = boto3.client('s3')
contents = conn.list_objects(Bucket=bucket, Prefix=sub)['Contents']
for f in contents:
    print(f['Key'])

nlp/
nlp/Test.txt


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


In [4]:

# Connect to S3 bucket and download file
s3 = boto3.resource('s3')
buck=s3.Bucket(bucket)
# for obj in buck.objects.all():
#     key = obj.key
#     text = obj.get()['Body'].read()
# text=text.decode("utf-8")

In [5]:
my_bucket = 'cogni-nlp-bucket'
my_file = 'nlp/Test.txt'
s3client = boto3.client('s3')
response = s3client.get_object(Bucket=my_bucket, Key=my_file)
body = response['Body'].read()
text=body.decode('cp1252')
text

'Indian Mutiny, also called Sepoy Mutiny or First War of Independence, widespread but unsuccessful rebellion against British rule in India in 1857–59. Begun in Meerut by Indian troops (sepoys) in the service of the British East India Company, it spread to Delhi, Agra, Kanpur, and Lucknow. In India it is often called the First War of Independence and other similar names.\r\nTo regard the rebellion merely as a sepoy mutiny is to underestimate the root causes leading to it. British paramountcy—i.e., the belief in British dominance in Indian political, economic, and cultural life—had been introduced in India about 1820. The British increasingly used a variety of tactics to usurp control of the Hindu princely states that were under what were called subsidiary alliances with the British. Everywhere the old Indian aristocracy was being replaced by British officials. One notable British technique was called the doctrine of lapse, first perpetrated by Lord Dalhousie in the late 1840s. It involv

In [6]:
import spacy

# Load the large English NLP model
nlp = spacy.load('en_core_web_lg')

# Parse the text with spaCy. This runs the entire pipeline.
# so we've already segmented and tokenized the doc in one line
doc = nlp(text)

#with this pipeline we can look at different kind of words and phrases
#for instance let us look at flat phrases that have a noun as their head
for chunk in doc.noun_chunks:
    print(chunk.text, chunk.root.text, chunk.root.dep_,
            chunk.root.head.text)

Indian Mutiny Mutiny nsubj called
Sepoy Mutiny Mutiny oprd called
First War War conj Mutiny
Independence Independence pobj of
widespread but unsuccessful rebellion rebellion appos Mutiny
British rule rule pobj against
India India pobj in
Meerut Meerut pobj in
Indian troops troops pobj by
sepoys sepoys appos troops
the service service pobj in
the British East India Company Company pobj of
it it nsubj spread
Delhi Delhi pobj to
Agra Agra conj Delhi
Kanpur Kanpur conj Agra
Lucknow Lucknow conj Kanpur
India India pobj In
it it nsubjpass called
the First War War oprd called
Independence Independence pobj of
other similar names names conj War
the rebellion rebellion dobj regard
a sepoy mutiny mutiny nsubj is
the root causes causes dobj underestimate
it it pobj to
British paramountcy paramountcy nsubjpass introduced
the belief belief appos paramountcy
British dominance dominance pobj in
Indian political, economic, and cultural life life pobj in
India India pobj in
The British British nsubj us

In [7]:
#as sentences

sents=text.split('.')
sents

['Indian Mutiny, also called Sepoy Mutiny or First War of Independence, widespread but unsuccessful rebellion against British rule in India in 1857–59',
 ' Begun in Meerut by Indian troops (sepoys) in the service of the British East India Company, it spread to Delhi, Agra, Kanpur, and Lucknow',
 ' In India it is often called the First War of Independence and other similar names',
 '\r\nTo regard the rebellion merely as a sepoy mutiny is to underestimate the root causes leading to it',
 ' British paramountcy—i',
 'e',
 ', the belief in British dominance in Indian political, economic, and cultural life—had been introduced in India about 1820',
 ' The British increasingly used a variety of tactics to usurp control of the Hindu princely states that were under what were called subsidiary alliances with the British',
 ' Everywhere the old Indian aristocracy was being replaced by British officials',
 ' One notable British technique was called the doctrine of lapse, first perpetrated by Lord D

In [8]:
#as words

for word in doc:
    print(word.text)

Indian
Mutiny
,
also
called
Sepoy
Mutiny
or
First
War
of
Independence
,
widespread
but
unsuccessful
rebellion
against
British
rule
in
India
in
1857–59
.
Begun
in
Meerut
by
Indian
troops
(
sepoys
)
in
the
service
of
the
British
East
India
Company
,
it
spread
to
Delhi
,
Agra
,
Kanpur
,
and
Lucknow
.
In
India
it
is
often
called
the
First
War
of
Independence
and
other
similar
names
.


To
regard
the
rebellion
merely
as
a
sepoy
mutiny
is
to
underestimate
the
root
causes
leading
to
it
.
British
paramountcy
—
i.e.
,
the
belief
in
British
dominance
in
Indian
political
,
economic
,
and
cultural
life
—
had
been
introduced
in
India
about
1820
.
The
British
increasingly
used
a
variety
of
tactics
to
usurp
control
of
the
Hindu
princely
states
that
were
under
what
were
called
subsidiary
alliances
with
the
British
.
Everywhere
the
old
Indian
aristocracy
was
being
replaced
by
British
officials
.
One
notable
British
technique
was
called
the
doctrine
of
lapse
,
first
perpetrated
by
Lord
Dalhousie
in
the


In [9]:
# now we'll filter out stop words
filter_doc=""
for token in doc:
  if token.is_stop:
    print(token.text)
  else:
    filter_doc+=token.text+ " "

also
or
First
of
but
against
in
in
in
by
in
the
of
the
it
to
and
In
it
is
often
the
First
of
and
other
To
the
as
a
is
to
the
to
it
the
in
in
and
had
been
in
about
The
used
a
of
to
of
the
that
were
under
what
were
with
the
Everywhere
the
was
being
by
One
was
the
of
first
by
in
the
It
the
a
without
a
from
a
and
after
the
or
his
To
those
may
be
the
of
the
many
of
whom
had
been
of
their
or
had
Another
serious
was
the
of
by
which
was
being
by
the
of
were
the
of
the
The
to
that
than
the
During
his
as
of
made
toward
and
had
a
to
all
to
the
of
to
were
to
with
their
in
the
of
the
There
was
a
that
the
at
down
the
The
of
of
was
a
to
both
and
The
out
in
the
because
it
was
only
in
the
that
were
The
for
was
the
of
the
To
it
the
had
to
off
the
of
A
among
the
that
the
used
to
the
was
a
of
and
thus
to
have
with
it
was
an
to
both
and
There
is
no
that
either
of
these
was
used
on
any
of
the
in
However
the
that
the
were
to
the
that
the
were
to
For
their
part
the
did
not
enough
to
the
of
In
a
at
the
in
He
w

In [11]:
#Since spacy does not have a built in lemmatizer, we'd build a lemmatizer instead

#Text Lemmatization: finding the base word of our word in doc
# for instance stand becomes standing, fell becomes fall
# we'll put lemmatized words in a new filtered doc
doc = nlp(filter_doc)
filter_doc=""
for token in doc:
  filter_doc+=token.lemma_ + " "
  if token.text != token.lemma_:
    print(token.text+" becomes "+token.lemma_)

Hindu becomes hindu
British becomes british
’ becomes '
’ becomes '


In [13]:
!mkdir results

In [19]:
import numpy as np
doc = nlp(filter_doc)
words=[]
with open('results/words.txt','w') as f:

    for word in doc:
        words.append(word.text)
        f.write(word.text)
        f.write('\n')
np.save('results/words.npy',words,allow_pickle=True)



In [22]:
import pickle
pickle.dump(words, open('results/words.pickle', 'wb'))

In [16]:
# no used
# import logging
# import boto3
# from botocore.exceptions import ClientError
# import os


# def upload_file(file_name, bucket,key=None, object_name=None):
#     """Upload a file to an S3 bucket

#     :param file_name: File to upload
#     :param bucket: Bucket to upload to
#     :param object_name: S3 object name. If not specified then file_name is used
#     :return: True if file was uploaded, else False
#     """

#     # If S3 object_name was not specified, use file_name
#     if object_name is None:
#         object_name = os.path.basename(file_name)
#     if key is None:
#         met={}
#     else:
#         met={key}

#     # Upload the file
#     s3_client = boto3.client('s3')
#     try:
#         response = s3_client.upload_file(file_name, bucket, object_name)
#     except ClientError as e:
#         logging.error(e)
#         return False
#     return True

In [21]:
s3 = boto3.client('s3')

folder_name = "results"

s3.put_object(Bucket=my_bucket, Key=(folder_name+'/'))

{'ResponseMetadata': {'RequestId': 'VNNEAW04W1W1FHDJ',
  'HostId': 'C0xSZq0QTBapbvrJagpvIUO4Eunp2aDm6Zx+vM62GDtYPX/IBeT30iRCyHAMaZqn5lbqN0s2jlU=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'C0xSZq0QTBapbvrJagpvIUO4Eunp2aDm6Zx+vM62GDtYPX/IBeT30iRCyHAMaZqn5lbqN0s2jlU=',
   'x-amz-request-id': 'VNNEAW04W1W1FHDJ',
   'date': 'Sun, 05 Jun 2022 06:05:35 GMT',
   'etag': '"d41d8cd98f00b204e9800998ecf8427e"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"d41d8cd98f00b204e9800998ecf8427e"'}

In [23]:
with open('results/words.npy','rb') as data:
    s3.put_object(Bucket=my_bucket,Key=(folder_name+'/'+'words.npy'),Body=data)
with open('results/words.txt','rb') as data:
    s3.put_object(Bucket=my_bucket,Key=(folder_name+'/'+'words.txt'),Body=data)
with open('results/words.pickle','rb') as data:
    s3.put_object(Bucket=my_bucket,Key=(folder_name+'/'+'words.pickle'),Body=data)